# Balanced score card

Realizado por:

-Jorge Pineda
-Andres Camargo
-Luis Palmera

In [1]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

class Logica:
    def __init__(self):
        # Import and display first five rows of advertising dataset
        self.advert = pd.read_csv('Advertising.csv')
        self.advert.head()
        self.sumtotal = self.advert['sales'].sum()
        self.mean = self.advert['sales'].mean()
        self.totalCost = self.advert['TV'].sum() + self.advert['radio'].sum() 
    
    def makeLinealRegression(self, adCostTV, adCostRadio):
        # Build linear regression model using TV and Radio as predictors
        # Split data into predictors X and output Y
        predictors = ['TV', 'radio']
        X = self.advert[predictors]
        y = self.advert['sales']

        # Initialise and fit model
        lm = LinearRegression()
        model = lm.fit(X, y)
        #model.predict(X)
        new_X = [[adCostTV, adCostRadio]]
        return model.predict(new_X)[0]
    def getMean(self):
        return self.mean
    def getTotal(self):
        return self.sumtotal
    def getTotalCost(self):
        return self.totalCost


In [2]:
import plotly.graph_objects as go
import threading

threadLock = threading.Lock()

class Figura(threading.Thread):
    def __init__(self, title = "gráfica", value=0, reference=250, filename = "image.png", maxrange = 500):
        #super().__init__()
        threading.Thread.__init__(self)
        self.value = value
        self.reference = reference
        self.range = maxrange
        self.title = title
        self.filename = filename
        #self.make()
        
    def run(self):
        threadLock.acquire()
        fig = go.Figure(go.Indicator(
            domain = {'x': [0, 1], 'y': [0, 1]},
            value = self.value,
            mode = "gauge+number+delta",
            title = {'text': self.title},
            delta = {'reference': self.reference},
            gauge = {'axis': {'range': [None, self.range]},
                     'bar':{'color':'dodgerblue'},
                     'steps' : [
                         {'range': [0, self.range/3], 'color': "red"},
                         {'range': [self.range/3, (self.range/3)*2], 'color': "orange"},
                         {'range': [(self.range/3)*2, self.range], 'color': "yellowgreen"}],
                     'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': self.reference}}))

        #fig.show()
        #import plotly.io as pio
        #pio.orca.config

        #pio.orca.config.executable = "/home/jorge/anaconda3/envs/myenv/bin/orca"
        #pio.orca.config.save()

        #from IPython.display import SVG, display
        #img_bytes = pio.to_image(fig, format="svg")
        #display(SVG(img_bytes))
        fig.write_image(self.filename)
        threadLock.release()
#fg = Figura()
#import plotly.io as pio
#pio.renderers

pip install plotly
conda install -c plotly plotly-orca

In [3]:
import sys
import psutil
import requests
import locale

from PyQt5.QtSvg import QSvgWidget, QSvgRenderer
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton, QHBoxLayout, QGridLayout, QLineEdit
from PyQt5.QtCore import pyqtSlot
from PyQt5.QtGui import QIcon, QPixmap, QIntValidator


class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'Balanced score card'
        self.left = 10
        self.top = 10
        self.width = 640
        self.height = 480
        self.nameImgSales = "sales.png"
        self.nameImgEstimate = "estimate.png"
        self.reference = 280
        self.adCostTV = 300
        self.adCostRadio = 200
        self.maxrange = 350
        self.value = self.makeLinealRegression()
        self.drawGraphics()
        self.initUI()
    
    def makeLinealRegression(self):
        log = Logica()
        self.maxrange = log.getMean()
        self.total = log.getTotal()
        self.totalCost = log.getTotalCost()
        return log.makeLinealRegression(float(self.adCostTV),float(self.adCostRadio))
        
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        #Texto
        lbDescripcion = QLabel(self)
        lbDescripcion.setText('Ventas de medios de comunicación (TV y Radio)')
        lbDescripcion.move(100,50)
        
        gridParametros = QHBoxLayout()
        gridParametros.addStretch(1)
        
        hBoxReferencia = QHBoxLayout()
        hBoxReferencia.addStretch(1)
        lbReferencia = QLabel()
        lbReferencia.setText("Referencia:")
        self.tBoxReferencia = QLineEdit()
        self.tBoxReferencia.setValidator(QIntValidator())
        self.tBoxReferencia.setMaxLength(8)
        self.tBoxReferencia.setText(str(self.reference))
        hBoxReferencia.addWidget(lbReferencia)
        hBoxReferencia.addWidget(self.tBoxReferencia)
        
        hBoxValorTV = QHBoxLayout()
        hBoxValorTV.addStretch(1)
        lbValorTV = QLabel()
        lbValorTV.setText("Gasto publicidad TV:")
        self.tBoxValorTV = QLineEdit()
        self.tBoxValorTV.setValidator(QIntValidator())
        self.tBoxValorTV.setMaxLength(8)
        self.tBoxValorTV.setText(str(self.adCostTV))
        hBoxValorTV.addWidget(lbValorTV)
        hBoxValorTV.addWidget(self.tBoxValorTV)
        
        hBoxValorRadio = QHBoxLayout()
        hBoxValorRadio.addStretch(1)
        lbValorRadio = QLabel()
        lbValorRadio.setText("Gasto publicidad Radio:")
        self.tBoxValorRadio = QLineEdit()
        self.tBoxValorRadio.setValidator(QIntValidator())
        self.tBoxValorRadio.setMaxLength(8)
        self.tBoxValorRadio.setText(str(self.adCostRadio))
        hBoxValorRadio.addWidget(lbValorRadio)
        hBoxValorRadio.addWidget(self.tBoxValorRadio)
        
        hBoxGanancia = QHBoxLayout()
        hBoxGanancia.addStretch(1)
        lbGanancia = QLabel()
        lbGanancia.setText("Ganancia esperada:")
        self.tBoxGanancia = QLabel()
        self.tBoxGanancia.setText(locale.currency( self.value, grouping=True ))
        hBoxValorRadio.addWidget(lbGanancia)
        hBoxValorRadio.addWidget(self.tBoxGanancia)
        
        gridParametros.addLayout(hBoxReferencia)
        gridParametros.addLayout(hBoxValorTV)
        gridParametros.addLayout(hBoxValorRadio)
        gridParametros.addLayout(hBoxGanancia)
        
        button = QPushButton('Calcular y refrescar gráfica', self)
        button.setToolTip('Presione aquí para ')
        button.move(100,70)
        button.clicked.connect(self.on_click)
        
        self.grid = QGridLayout()
        self.grid.addWidget(lbDescripcion, 0, 0, 1, 2)
        self.grid.addWidget(self.showGraphic(self.nameImgSales), 1, 0)
        self.grid.addWidget(self.showGraphic(self.nameImgEstimate), 1, 1)
        
        self.grid.addLayout(gridParametros, 2, 0, 1, 2)
        self.grid.addWidget(button, 3, 0, 1, 2)
        
        self.lbEstado = QLabel()
        self.lbEstado.setText("-")
        self.grid.addWidget(self.lbEstado, 4, 0, 1, 2)
        
        #grid.addWidget(txtPass, 1, 1)
        self.setLayout(self.grid)
        
        self.show()
        
    def drawGraphics(self):
        threads = []
        
        t = Figura("Ventas", self.total, self.total/2, self.nameImgSales, self.totalCost)
        t2 = Figura("Proyección de ventas",self.value,self.reference, self.nameImgEstimate, self.total)
        
        t.start()
        t2.start()
        
        # Add threads to thread list
        threads.append(t)
        threads.append(t2)

        print("Pasó start")
        
        # Wait for all threads to complete
        for t in threads:
            t.join()
        print("Pasó join")
            
    def showGraphic(self, filename):
        # Create widget
        label = QLabel(self)
        pixmap = QPixmap(filename)
        #pixmap = fig.show(renderer="png");
        label.setPixmap(pixmap)
        self.resize(pixmap.width(),pixmap.height())
        return label
    
    @pyqtSlot()
    def on_click(self):
        self.lbEstado.setText("Cargando...")
        
        self.adCostTV = float(self.tBoxValorTV.text())
        self.adCostRadio = float(self.tBoxValorRadio.text())
        self.value = self.makeLinealRegression()
        self.tBoxGanancia.setText(locale.currency( self.value, grouping=True ))
        self.reference = float(self.tBoxReferencia.text())
        self.drawGraphics()
        self.grid.addWidget(self.showGraphic(self.nameImgSales), 1, 0)
        self.grid.addWidget(self.showGraphic(self.nameImgEstimate), 1, 1)
        self.grid.update()
        
        
        self.lbEstado.setText("-")
        
if __name__ == '__main__':
    
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

Pasó start
Pasó join
Pasó start
Pasó join
Pasó start
Pasó join


SystemExit: 0

/home/jorge/anaconda3/envs/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

